In [5]:
# Import
import requests
from configparser import ConfigParser
import pandas as pd
from typing import List

from ns_api import npi_res
from pyspark import pandas as ppd

# class FinderWrapper:
#     def __init__(self, finder):
#         self.finder = finder

#     def find_spec(self, fullname, path, target=None):
#         print(f"Finder {self.finder} checking for module: {fullname}, path: {path}")
#         return self.finder.find_spec(fullname, path, target)

# Wrap all meta path finders
# sys.meta_path = [FinderWrapper(finder) for finder in sys.meta_path]

c:\Users\PC\anaconda3\envs\env\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [6]:
# Reload lib
from importlib import reload

npi_res = reload(npi_res)

In [7]:
# Get secrets
config = ConfigParser()
config.read("../config.cfg")

['../config.cfg']

In [64]:
# Define the API endpoint and parameters
subscription_key = config["ns-api-token"]["pri_key"]
headers = {
    # Request headers
    "Ocp-Apim-Subscription-Key": subscription_key,
}
lang = "en"
originLat = 51.451472
originLng = 5.473389
destinationLat = 52.379389
destinationLng = 4.899389
originWalk = "true"
originBike = "true"
destinationWalk = "true"
destinationBike = "true"

# Trips API
url = (
    # f"https://gateway.apiportal.ns.nl/reisinformatie-api/api/v3/trips" +
    # f"https://gateway.apiportal.ns.nl/reisinformatie-api/api/v2/departures" +
    f"https://gateway.apiportal.ns.nl/nsapp-stations/v2"
    +
    # f"?" +
    # f"q={r'Eindhoven Cen'}"
    # f"{lang}&" +
    # f"{fromStation}&" +
    # f"originLat={originLat}&" +
    # f"originLng={originLng}&" +
    # f"destinationLat={destinationLat}&" +
    # f"destinationLng={destinationLng}&" +
    # f"{lang}&" +
    # f"{lang}&" +
    f""
)
print(url)
# Send the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code in [200]:
    data = response.json()  # Parse the JSON response
    # print(data)  # Print the response data
else:
    print(f"Error: {response.status_code}, {response.text}")

https://gateway.apiportal.ns.nl/nsapp-stations/v2


In [6]:
# Parsing
stations = [npi_res.Station(stat_dict) for stat_dict in data["payload"]]
stations_df = pd.DataFrame(data["payload"])
pstations_df = ppd.DataFrame(data["payload"])

In [9]:
# Func 
def objects_to_dataframe(obj_lst : List[object]) -> pd.DataFrame:
    return pd.DataFrame([obj.__dict__ for obj in obj_lst])

stations_df = objects_to_dataframe(stations)
stations_df.to_parquet("stations_df.parquet")

c:\Users\PC\anaconda3\envs\env\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


#### Trip APIs
-------------------

In [8]:
# Read station info 
stations_df = pd.read_parquet("../outputs/stations_df.parquet")

In [50]:
# Define the API endpoint and parameters
subscription_key = config["ns-api-token"]["pri_key"]
headers = {
    # Request headers
    "Ocp-Apim-Subscription-Key": subscription_key,
}
lang = "en"
originLat = 51.451472
originLng = 5.473389
destinationLat = 52.379389
destinationLng = 4.899389
originWalk = "true"
originBike = "true"
destinationWalk = "true"
destinationBike = "true"
fromStation = "EHV"
toStation = "ASA"
samp_q = "zuideramstel".replace(",", " ")
# Trips API
url = (
    # f"https://gateway.apiportal.ns.nl/reisinformatie-api/api/v3/trips" +
    # f"https://gateway.apiportal.ns.nl/reisinformatie-api/api/v2/departures" +
    # f"https://gateway.apiportal.ns.nl/nsapp-stations/v2" +
    f"https://gateway.apiportal.ns.nl/places-api/v2/places?q={samp_q}" + 
    # f"?" +
    # f"q={r'Eindhoven Cen'}"
    # f"{lang}&" +
    # f"fromStation={fromStation}&" +
    # f"toStation={toStation}" +
    # f"originLat={originLat}&" +
    # f"originLng={originLng}&" +
    # f"destinationLat={destinationLat}&" +
    # f"destinationLng={destinationLng}&" +
    # f"{lang}&" +
    # f"{lang}&" +
    f""
)
print(url)
# Send the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code in [200]:
    data = response.json()  # Parse the JSON response
    # print(data)  # Print the response data
else:
    print(f"Error: {response.status_code}, {response.text}")

https://gateway.apiportal.ns.nl/places-api/v2/places?q=zuideramstel


In [51]:
data

{'links': {}, 'payload': [], 'meta': {}}

#### Get Place based on query
------------

In [11]:
# Define the API endpoint and parameters
subscription_key = config["ns-api-token"]["pri_key"]
headers = {
    # Request headers
    "Ocp-Apim-Subscription-Key": subscription_key,
}
lang = "en"
originLat = 51.451472
originLng = 5.473389
destinationLat = 52.379389
destinationLng = 4.899389
originWalk = "true"
originBike = "true"
destinationWalk = "true"
destinationBike = "true"
fromStation = "EHV"
toStation = "ASA"
q = "Zuideramstel, Amsterdam"
prompt = "Zuideramstel, Amsterdam"
# Trips API
url = (
    # f"https://gateway.apiportal.ns.nl/autosuggest-api/locaties?q=Zuiderlandseweg" +
    f"https://gateway.apiportal.ns.nl/places-api/v3/nearbyme/ai?prompt={prompt}" + 
    # f"https://gateway.apiportal.ns.nl/nsapp-stations/v2" +
    # f"?" +
    # f"q={r'Eindhoven Cen'}"
    # f"q={q}" +
    # f"fromStation={fromStation}&" +
    # f"toStation={toStation}" +
    # f"originLat={originLat}&" +
    # f"originLng={originLng}&" +
    # f"destinationLat={destinationLat}&" +
    # f"destinationLng={destinationLng}&" +
    # f"{lang}&" +
    # f"{lang}&" +
    f""
)
print(url)
# Send the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code in [200]:
    data = response.json()  # Parse the JSON response
    # print(data)  # Print the response data
else:
    print(f"Error: {response.status_code}, {response.text}")

https://gateway.apiportal.ns.nl/places-api/v3/nearbyme/ai?prompt=Zuideramstel, Amsterdam
Error: 404, { "statusCode": 404, "message": "Resource not found" }


In [12]:
# Def pull places
def extract_ns_place(query, headers, *wargs, **kwargs):
    query = query.replace(",", "")
    url = (
    f"https://gateway.apiportal.ns.nl/places-api/v2/places" + 
    f"?" +
    f"q={query}"
    f""
)
    response = requests.get(url, headers=headers)

    if response.status_code in [200]:
        data = response.json()  
    else:
        print(f"Error: {response.status_code}, {response.text}")

    return data 

In [13]:
# Import housing df
housing_df = pd.read_pickle("../outputs/all_listings_.pkl")

In [14]:
# Test query search place
r = map(lambda x : extract_ns_place(x, headers=headers), housing_df["location"].unique())
r = [data["payload"] for data in r]
return_count = [len(d) for d in r]


In [31]:
r[-2][-5]["locations"]

[{'name': 'Utrecht, Rubenslaan',
  'lat': 52.07938173665342,
  'lng': 5.136382737192676,
  'open': 'Unknown',
  'link': {'uri': 'https://places.ns-mlab.nl/api/v2/places/stop/NL:S:50001030'},
  'extra': {'code': 'NL:S:50001030',
   'stopType': 'ONSTREET_BUS',
   'type': 'BUS_STOP',
   'routeShortNames': ''},
  'infoImages': [],
  'apps': [],
  'sites': [],
  'extraInfo': [],
  'nearbyMeLocationId': {'value': 'NL:S:50001030', 'type': 'stop'}},
 {'name': 'Utrecht, CS Centrumzijde',
  'lat': 52.08915125115162,
  'lng': 5.111975483114268,
  'open': 'Unknown',
  'link': {'uri': 'https://places.ns-mlab.nl/api/v2/places/stop/NL:S:50000001'},
  'extra': {'code': 'NL:S:50000001',
   'stopType': 'ONSTREET_TRAM',
   'type': 'BUS_STOP',
   'routeShortNames': '20,21,22'},
  'infoImages': [],
  'apps': [],
  'sites': [],
  'extraInfo': [],
  'nearbyMeLocationId': {'value': 'NL:S:50000001', 'type': 'stop'}},
 {'name': 'Utrecht, 24 Oktoberplein',
  'lat': 52.08161864333117,
  'lng': 5.093114961018972,


In [20]:
r[-2][-5].keys()

dict_keys(['type', 'name', 'nearbyMeMarkerIconName', 'identifiers', 'locations', 'open', 'keywords', 'stationBound', 'advertImages', 'infoImages'])

In [45]:
dict(zip(housing_df["location"].unique(), return_count))

{'Amsterdam Zuideramstel, Amsterdam': 0,
 'Amsterdam Westpoort, Amsterdam': 1,
 'Amsterdam Amsterdam-Zuidoost, Amsterdam': 0,
 'Amsterdam Centrum, Amsterdam': 2,
 'Amsterdam Zeeburg, Amsterdam': 1,
 'Amsterdam Geuzenveld / Slotermeer, Amsterdam': 0,
 'Amsterdam Oud-Zuid, Amsterdam': 0,
 'Amsterdam Slotervaart, Amsterdam': 0,
 'Amsterdam Oost-Watergraafsmeer, Amsterdam': 0,
 'Amsterdam Oud-West, Amsterdam': 0,
 'Amsterdam Noord, Amsterdam': 2,
 'Amsterdam De Baarsjes, Amsterdam': 0,
 'Amsterdam Westerpark, Amsterdam': 0,
 'Amsterdam Bos & Lommer, Amsterdam': 2,
 'Amsterdam Osdorp, Amsterdam': 1,
 'Amsterdam': 45,
 'Utrecht Overvecht, Utrecht': 13,
 'Utrecht Binnenstad, Utrecht': 0,
 'Utrecht Zuid-West, Utrecht': 0,
 'Utrecht Noord-Oost, Utrecht': 0,
 'Utrecht Zuid, Utrecht': 1,
 'Utrecht Noord-West, Utrecht': 0,
 'Utrecht Leidsche Rijn, Utrecht': 11,
 'Utrecht Oost, Utrecht': 2,
 'Utrecht West, Utrecht': 3,
 'Utrecht': 37,
 'Eindhoven, North Brabant': 0}

In [ ]:
# 